In [1]:
using ParallelTemperingMonteCarlo
# using Pkg;Pkg.add("BenchmarkTools")
using BenchmarkTools
using Random,StaticArrays

# Equilibration

The purpose of this notebook is to write functions aimed at equilibrating the ptmc run. It must iterate over every trajectory for, by default 20\% of the total cycles run with _no_ sampling, other than testing the $\geq$ condition to update the min/max energies

In [27]:
Random.seed!(1234)

# number of atoms
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details

mc_cycles = 1000000 #default 20% equilibration cycles on top

mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[])

first, the $\geq$ condition: test whether it's faster to unpack mc_states or not. 

In [3]:
function testmin(new_en,en_min)
    if new_en < en_min
        en_min = new_en
    end
    return en_min
end

function test_min(new_en,en_min)
    return ifelse(new_en<en_min,new_en,en_min)
end
function test_max(new_en,en_max)
    return ifelse(new_en>en_max,new_en,en_max)
end

test_max (generic function with 1 method)

In [4]:
@benchmark testmin($rand(),$rand())

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.511 ns … 4.640 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.523 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.542 ns ± 0.085 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▇▇██▆▄  ▂▂▁▁▁       ▃▄▅▅▄▃                ▁▁             ▂
  ██████████████▅▆▆▃▅▄▆██████▇▅▇▆▆▇▇▆▆▆▅▆▆▆▆▆██▇█▆▅▄▆▅▆▇▆▇▆ █
  2.51 ns     Histogram: log(frequency) by time     2.66 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [5]:
@benchmark test_min($rand(),$rand())

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.460 ns … 4.802 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.517 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.518 ns ± 0.100 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▂▆█▆               ▃▃                                     
  ▂████▆▂▁▁▁▁▁▁▁▁▁▁▂▅▇███▃▁▁▁▁▁▁▁▁▁▁▁▁▃▇▅▇▇▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  2.46 ns        Histogram: frequency by time       2.63 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

Use the ifelse version, it's slightly faster more often. NB the function guarantees type stability, telling the compiler what is to be returned, this is better.

In [6]:
function check_e_bounds(energy,ebounds::Vector)
    ebounds[1] = test_min(energy,ebounds[1])
    ebounds[2] = test_max(energy,ebounds[2])
    return ebounds
end
function checkbounds2(energy,emin,emax)
    emin = test_min(energy,emin)
    emax = test_max(energy,emax)
    return emin,emax
end
function checkbounds3(energy,ebounds)
    if energy<ebounds[1]
        ebounds[1]=energy
    elseif energy>ebounds[2]
        ebounds[2] = energy
    else
    end
    return ebounds
end
function checkbounds4(energy,ebounds)
    ebounds[1],ebounds[2] = test_min(energy,ebounds[1]),test_max(energy,ebounds[2])
    return ebounds 
end

checkbounds4 (generic function with 1 method)

In [7]:
ebounds = [-1.,1.]
energy = 0.6
Random.seed!(1234)
ebounds = check_e_bounds(energy,ebounds)
@benchmark check_e_bounds($rand(-1.5:1.5),$ebounds)

BenchmarkTools.Trial: 10000 samples with 960 evaluations.
 Range (min … max):  89.855 ns … 120.740 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     92.985 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   93.170 ns ±   1.862 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

         ▃▅▆█▆▅▃▁     ▁▃▄▆▇▆▅▄▂▁         ▁                      
  ▁▁▂▂▄▆██████████▇██████████████▇▇▆▆▇▇▇██▇█▇▆▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂ ▅
  89.9 ns         Histogram: frequency by time         97.6 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [8]:
ebounds = [-1.,1.]
Random.seed!(1234)
@benchmark checkbounds2($rand(-1.5:1.5),$ebounds[1],$ebounds[2])

BenchmarkTools.Trial: 10000 samples with 955 evaluations.
 Range (min … max):  91.133 ns … 106.255 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     94.791 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   94.972 ns ±   1.816 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▁▄▃▃▁       ▁▂▄█▆▆▄▂▁        ▂▂▂▁                    
  ▁▁▁▂▂▂▄▅▇██████▇▆▆▅▆██████████▇▇▆▆▆▆▇██████▆▆▅▅▃▄▃▃▃▂▃▂▂▂▂▂▂ ▄
  91.1 ns         Histogram: frequency by time         99.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
ebounds = [-1.,1.]
Random.seed!(1234)
energy = 0.6
@benchmark checkbounds3($rand(-1.5:1.5),$ebounds)

BenchmarkTools.Trial: 10000 samples with 956 evaluations.
 Range (min … max):  89.738 ns … 122.505 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     93.431 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   93.563 ns ±   1.947 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▁▂▄▅▂▃▁    ▁▃▂▅▆█▇▅▄▃  ▁ ▂▃▄▅▅▆▆▇▄▂                     
  ▁▂▃▃▄██████████▇████████████████████████████▇▇▇▆▅▄▄▄▃▃▃▂▂▂▂▂ ▅
  89.7 ns         Histogram: frequency by time           98 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [10]:
ebounds = [-1.,1.]
Random.seed!(1234)
energy = 0.6
ebounds = checkbounds4(energy,ebounds)
@benchmark checkbounds4($rand(-1.5:1.5),$ebounds)

BenchmarkTools.Trial: 10000 samples with 957 evaluations.
 Range (min … max):  89.764 ns … 107.478 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     94.510 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   94.286 ns ±   2.076 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

          ▁▁        ▁▄▄▅▃▁      ▃▄▆██▇▅▃▃▁                      
  ▁▁▁▂▄▄▆████▇▇▆▆▆▆█████████▆▆▇███████████▇█▇▇▆▆▆▆▅▄▄▅▄▃▃▂▃▃▂▂ ▅
  89.8 ns         Histogram: frequency by time           99 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [11]:
import ParallelTemperingMonteCarlo.MCRun.mc_cycle!,ParallelTemperingMonteCarlo.MCRun.mc_step!,ParallelTemperingMonteCarlo.MCRun.update_max_stepsize!

function test_equilibration(mc_states,move_strat,mc_params,pot,ensemble)#more booleans etc in existing code. Will include after testing


    a,v,r = atom_move_frequency(move_strat),vol_move_frequency(move_strat),rot_move_frequency(move_strat)
    n_steps = a + v + r
    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i = 1:mc_params.eq_cycles
        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
        
        for state in mc_states
            ebounds = checkbounds3(state.en_tot,ebounds)
        end

        if rem(i, mc_params.n_adjust) == 0
            for i_traj = 1:mc_params.n_traj
                update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
            end 
        end
    end
    return mc_states,ebounds,a,v,r
end

test_equilibration (generic function with 1 method)

In [17]:
@benchmark test_equilibration($mc_states,move_strat,mc_params,pot,ensemble)

Total number of moves per MC cycle: 13



Total number of moves per MC cycle: 13



Total number of moves per MC cycle: 13



Total number of moves per MC cycle: 13



BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 19.070 s (5.66% GC) to evaluate,
 with a memory estimate of 30.72 GiB, over 179400092 allocations.

NB: Tested the three "sensible" methods for checking the energy and the fastest overall was iteration checkbounds3, likely as a result of it not calculating max/min if the other is satisfied first. The major quesiton is which goes first? This could have a huge impact on how quick the process is: Testing time:

In [13]:
function checkbounds3a(energy,ebounds)
    #if energy<ebounds[1]
        # ebounds[1]=energy
    if energy>ebounds[2]
        ebounds[2] = energy
    elseif energy<ebounds[1]
        ebounds[1]=energy
    else
    end
    return ebounds
end

checkbounds3a (generic function with 1 method)

In [14]:
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot) for i in 1:n_traj]

function test_equilibration2(mc_states,move_strat,mc_params,pot,ensemble)#more booleans etc in existing code. Will include after testing


    a,v,r = atom_move_frequency(move_strat),vol_move_frequency(move_strat),rot_move_frequency(move_strat)
    n_steps = a + v + r
    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i = 1:mc_params.eq_cycles
        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
        
        for state in mc_states
            ebounds = checkbounds3a(state.en_tot,ebounds)
        end

        if rem(i, mc_params.n_adjust) == 0
            for i_traj = 1:mc_params.n_traj
                update_max_stepsize!(mc_states[i_traj], mc_params.n_adjust, a, v, r)
            end 
        end
    end
    return mc_states,ebounds,a,v,r
end

test_equilibration2 (generic function with 1 method)

In [16]:
@benchmark test_equilibration2($mc_states,move_strat,mc_params,pot,ensemble)

Total number of moves per MC cycle: 13



Total number of moves per MC cycle: 13



Total number of moves per MC cycle: 13



Total number of moves per MC cycle: 13



BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 19.204 s (5.71% GC) to evaluate,
 with a memory estimate of 30.72 GiB, over 179400093 allocations.

# Equilibration complete



In [26]:
function reset_counters(state)
    state.count_atom = [0,0]
    state.count_vol = [0,0]
    state.count_rot = [0,0]
    state.count_exc = [0,0]
end

function test_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)#more booleans etc in existing code. Will include after testing


    a,v,r = atom_move_frequency(move_strat),vol_move_frequency(move_strat),rot_move_frequency(move_strat)
    n_steps = a + v + r
    println("Total number of moves per MC cycle: ", n_steps)
    println()

    ebounds = [100. , -100.]

    for i = 1:mc_params.eq_cycles

        mc_states = mc_cycle!(mc_states,move_strat,mc_params,pot,ensemble,n_steps,a,v,r)
        
        for state in mc_states
            ebounds = checkbounds3(state.en_tot,ebounds)
        end

        if rem(i, mc_params.n_adjust) == 0
            for state in mc_states
                update_max_stepsize!(state,mc_params.n_adjust,a,v,r)
            end
        end
        
    end

    #reset counter-variables
    for state in mc_states
        reset_counters(state)
    end

    delta_en_hist,delta_r2 = initialise_histograms!(mc_params,results,ebounds,mc_states[1].config.bc)

    start_counter = 1

    println("equilibration done")

    return mc_states,results,delta_en_hist,delta_r2,start_counter,a,v,r

end

test_equilibration (generic function with 2 methods)

In [22]:
function bench_test1(mc_states,mc_params)
    for i_traj = 1:mc_params.n_traj
        reset_counters(mc_states[i_traj])
    end
    return mc_states
end
function bench_test2(mc_states)
    for state in mc_states
        reset_counters(state)
    end
    return mc_states
end
@benchmark bench_test1(mc_states,mc_params)
@benchmark bench_test2(mc_states)

bench_test2 (generic function with 1 method)

It seems for state in states is better than iterating 1:mc_cycles

In [28]:
mc_states,results,delta_en_hist,delta_r2,start_counter,a,v,r = test_equilibration(mc_states,move_strat,mc_params,pot,ensemble,results)

Total number of moves per MC cycle: 13



equilibration done


(MCState{Float64, 13, SphericalBC{Float64}}[MCState{Float64, 13, SphericalBC{Float64}}(5.0, 63154.997052619445, Config{13, SphericalBC{Float64}, Float64}(SVector{3, Float64}[[-2.253745975907271, 0.8809630451693, -4.65183579949347], [-6.539847212461429, 2.604394415463834, -0.24223396420503163], [-1.2993256601676821, 5.389861763178107, 0.2108883631524374], [-2.8515737132492776, 3.221902337542295, 5.112981980059696], [2.982321513180388, 2.768210429993844, 3.276964238930853], [-6.541008974625659, -1.194603086794014, 4.477816180891731], [-0.21208827676342543, -4.318613312121051, -2.692140981463342], [-0.0595386777784796, -1.3325351768209939, 6.748768510773379], [-5.927068338421679, -3.582706545987274, -1.015661150134257], [3.3898556624907346, 0.056561854943967455, -1.8560933770436991], [2.8779737567608557, -3.871942153282416, 2.393149292969535], [-1.4878414715811006, -0.4651854251002149, 1.0510135367557591], [-2.331770616880134, -5.543205257778424, 3.2558197463258467]], SphericalBC{Float64}

-0.0022021688273434928